In [18]:
import torch

In [19]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-559.9733, 1102.8076,  327.2597], grad_fn=<MulBackward0>)


In [20]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)


In [21]:
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

net.zero_grad()
out.backward(torch.randn(1, 10))


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
tensor([[-0.1246, -0.0072,  0.0400,  0.0386,  0.0846, -0.0852, -0.0241, -0.0056,
          0.0550, -0.0742]], grad_fn=<ThAddmmBackward>)


In [25]:





output = net(input)
target = torch.randn(10)  # a dummy target, for example
print(target)

tensor([-0.1065, -0.2712, -0.2808, -0.7733,  0.7559, -2.0392,  0.5815,  0.2623,
         2.0072, -1.8140])


In [26]:
print(target.view(-1,1))

tensor([[-0.1065],
        [-0.2712],
        [-0.2808],
        [-0.7733],
        [ 0.7559],
        [-2.0392],
        [ 0.5815],
        [ 0.2623],
        [ 2.0072],
        [-1.8140]])


In [28]:
target = target.view(-1,1)

In [29]:

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


tensor(1.3225, grad_fn=<MseLossBackward>)


In [32]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

print(x)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [33]:
print(x)

tensor([-0.5468,  1.0770,  0.3196], requires_grad=True)


In [34]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [35]:
print(x)

tensor([-0.5468,  1.0770,  0.3196], requires_grad=True)


In [36]:
print(loss)

tensor(1.3221, grad_fn=<MseLossBackward>)
